In [1]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

## 데이터 확인

In [12]:
# 파일 불러오기

train = pd.read_csv('./open/word.csv', encoding='utf-8')
test_x = pd.read_csv('./open/test_x.csv', encoding='utf-8')
submission = pd.read_csv('./open/sample_submission.csv', encoding='utf-8')

In [13]:
# //// nltk stopwords 확인 / svd 예측실패한 텍스트 중에 단어수가 50 이상인 것들 stopwords에 추가(odin은 제외)
# 이건 내 주피터에서만 보이는거고 /////// 밑으로 추가한 것들

from nltk.corpus import stopwords
stop = stopwords.words('english')
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [14]:
# stopwords 적용
train.text = train.text.astype('str').str.split(' ')
train['text'] = train['text'].apply(lambda x: [word for word in x if word not in (stop)])
train['text'] = train['text'].apply(lambda x : (' ').join(x))
train

,text,author
0,almost choking much much wanted strange exclam...,3
1,sister suppose,2
2,engaged one walked perusing jane last letter d...,1
3,captain porch keeping carefully treacherous sh...,4
4,mercy gentlemen odin flung write anyway shame...,3
...,...,...
54874,smith odin whispered hardly dared hope would,2
54875,plan captain us settled details accomplishment,4
54876,sincere well wisher friend sister lucy odin,1
54877,wanted lend money,3


In [15]:
X = train.loc[:, 'text']
y = train.loc[:, 'author']

## 전처리

### train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

### 벡터라이즈

#### tfidfvectorizer

In [17]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환.
tfidf_vect = CountVectorizer(stop_words='english')
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

#### Multinomial Naive Bayes 적용

In [6]:
from sklearn.naive_bayes import MultinomialNB
mu_clf = MultinomialNB().fit(X_train_tfidf_vect, y_train)

In [7]:
# 정식으로 pipeline을 만들고

from sklearn.pipeline import Pipeline

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())
])

In [8]:
# 학습 후 train, test accuracy score 적용

from sklearn.metrics import accuracy_score

mu_clf.fit(X_train_tfidf_vect, y_train)

train_pred = mu_clf.predict(X_train_tfidf_vect)
test_pred = mu_clf.predict(X_test_tfidf_vect)

print('MultinomialNB train accuracy score:', accuracy_score(y_train, train_pred))
print('MultinomialNB test accuracy score:', accuracy_score(y_test, test_pred))
# predicted = text_clf.predict(X_test)

# np.mean(predicted == y_test)


MultinomialNB train accuracy score: 0.7595152950823406
MultinomialNB test accuracy score: 0.6868622448979592


In [9]:
from sklearn.metrics import classification_report

# print(confusion_matrix(y_test, pred))
# print('-'*50)
print(classification_report(y_train, train_pred))
print('-----------'*5)
print(classification_report(y_test, test_pred))

              precision    recall  f1-score   support

           0       0.73      0.85      0.79     10512
           1       0.94      0.59      0.73      5841
           2       0.85      0.68      0.76      9222
           3       0.66      0.96      0.78     12034
           4       0.97      0.49      0.65      6294

    accuracy                           0.76     43903
   macro avg       0.83      0.72      0.74     43903
weighted avg       0.80      0.76      0.75     43903

-------------------------------------------------------
              precision    recall  f1-score   support

           0       0.67      0.79      0.72      2723
           1       0.89      0.50      0.64      1381
           2       0.76      0.57      0.65      2332
           3       0.61      0.92      0.73      3029
           4       0.93      0.37      0.53      1511

    accuracy                           0.69     10976
   macro avg       0.77      0.63      0.66     10976
weighted avg       0.

#### 여러 모델들 적용

In [18]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV, SGDClassifier
from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier)
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


models = []
models.append(('LogisticRegression', LogisticRegression(random_state=13)))
models.append(('MultinomialNB', MultinomialNB()))
models.append(('RandomForestClassifier', RandomForestClassifier(random_state=13, n_jobs=-1)))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier(random_state=13)))
models.append(('AdaBoostClassifier', AdaBoostClassifier(random_state=13)))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier(random_state=13)))
models.append(('LGBMClassifier', LGBMClassifier(random_state=13)))
models.append(('KNeighborsClassifier', KNeighborsClassifier(n_neighbors=5, n_jobs=-1)))
models.append(('LinearSVC', LinearSVC(C=1, loss='hinge', random_state=13)))
models.append(('XgBoost', XGBClassifier(learning_rate=0.1, max_depth=3, random_state=13, n_jobs=-1)))
models.append(('RidgeClassifier', RidgeClassifier(random_state=13)))
models.append(('SGDClassifier', SGDClassifier(random_state=13, loss='modified_huber')))
# models.append(('RidgeClassifierCV', RidgeClassifierCV(cv=3)))

In [11]:
models

[('LogisticRegression', LogisticRegression(random_state=13)),
 ('MultinomialNB', MultinomialNB()),
 ('RandomForestClassifier',
  RandomForestClassifier(n_jobs=-1, random_state=13)),
 ('DecisionTreeClassifier', DecisionTreeClassifier(random_state=13)),
 ('AdaBoostClassifier', AdaBoostClassifier(random_state=13)),
 ('GradientBoostingClassifier', GradientBoostingClassifier(random_state=13)),
 ('LGBMClassifier', LGBMClassifier(random_state=13)),
 ('KNeighborsClassifier', KNeighborsClassifier(n_jobs=-1)),
 ('LinearSVC', LinearSVC(C=1, loss='hinge', random_state=13)),
 ('XgBoost',
  XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
                colsample_bynode=None, colsample_bytree=None, gamma=None,
                gpu_id=None, importance_type='gain', interaction_constraints=None,
                learning_rate=0.1, max_delta_step=None, max_depth=3,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                n_estimators=100, n_jobs=-

In [ ]:
from sklearn.metrics import accuracy_score

train_score = []
test_score = []
names = []

for name, model in models:
    clf = model
    clf.fit(X_train_tfidf_vect, y_train)
    
    train_pred = clf.predict(X_train_tfidf_vect)
    test_pred = clf.predict(X_test_tfidf_vect)
    
    names.append(name)
    train_score.append(accuracy_score(y_train, train_pred))
    test_score.append(accuracy_score(y_test, test_pred))

In [ ]:
result = pd.DataFrame({'model name': names,
                       'train score': train_score,
                       'test score': test_score})
result['diff'] = result['train score'] - result['test score']
result.round(2).sort_values(by='train score', ascending=False).reset_index(drop=True)

## 모델 하이퍼파라미터 튜닝

In [50]:
tfidf_vect = CountVectorizer(stop_words='english')
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

### MultinomialNB

In [51]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

params = {'alpha': [0.01, 0.1, 0.5, 1.0],
         'fit_prior': ['True', 'False']}
clf = MultinomialNB()
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_cv.fit(X_train_tfidf_vect, y_train)
print('MultinomialNB best parameters:', grid_cv.best_params_)
print('MultinomialNB best accuracy score:', grid_cv.best_score_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


MultinomialNB best parameters: {'alpha': 0.1, 'fit_prior': 'True'}
MultinomialNB best accuracy score: 0.6973555459276265


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    1.8s finished


In [52]:
mu_clf = MultinomialNB(alpha=0.1, fit_prior='True')
mu_clf.fit(X_train_tfidf_vect, y_train)

mu_train_pred = mu_clf.predict(X_train_tfidf_vect)
mu_test_pred = mu_clf.predict(X_test_tfidf_vect)

print('MultinomialNB train accuracy score', accuracy_score(y_train, mu_train_pred))
print('MultinomialNB test accuracy score', accuracy_score(y_test, mu_test_pred))

MultinomialNB train accuracy score 0.7898093524360522
MultinomialNB test accuracy score 0.7090014577259475


In [53]:
from sklearn.model_selection import StratifiedKFold

skfold = StratifiedKFold(n_splits=5)

In [54]:
from sklearn.model_selection import cross_validate
cross_validate(mu_clf, X_train_tfidf_vect, y_train, scoring=None, cv=skfold, return_train_score=True)

{'fit_time': array([0.01744914, 0.01574397, 0.01484299, 0.01434803, 0.01811409]),
 'score_time': array([0.00300908, 0.00269103, 0.00261188, 0.00249028, 0.00424099]),
 'test_score': array([0.70606992, 0.70276734, 0.69866758, 0.70615034, 0.70649203]),
 'train_score': array([0.79468709, 0.7950857 , 0.79693639, 0.79500612, 0.79452211])}

### LinearSVC

In [55]:
import sklearn
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

svc_clf = LinearSVC(C=1, loss='hinge', random_state=13)
svc_clf.fit(X_train_tfidf_vect, y_train)

svc_train_pred = svc_clf.predict(X_train_tfidf_vect)
svc_test_pred = svc_clf.predict(X_test_tfidf_vect)

print('LinearSVC train accuracy score:', accuracy_score(y_train, svc_train_pred))
print('LinearSVC test accuracy score:', accuracy_score(y_test, svc_test_pred))

LinearSVC train accuracy score: 0.8779582260893333
LinearSVC test accuracy score: 0.6758381924198251


In [56]:
params = {
    'C': [0.001, 0.01, 1], 
    'loss': ['squared_hinge', 'hinge'], 
    'penalty': ['l1', 'l2'], 
    'random_state': [13]
    }

grid_cv = GridSearchCV(LinearSVC(), param_grid=params, refit=True, verbose=2)
grid_cv.fit(X_train_tfidf_vect, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] C=0.001, loss=squared_hinge, penalty=l1, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l1, random_state=13, total=   0.0s
[CV] C=0.001, loss=squared_hinge, penalty=l1, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l1, random_state=13, total=   0.0s
[CV] C=0.001, loss=squared_hinge, penalty=l1, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l1, random_state=13, total=   0.0s
[CV] C=0.001, loss=squared_hinge, penalty=l1, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l1, random_state=13, total=   0.0s
[CV] C=0.001, loss=squared_hinge, penalty=l1, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l1, random_state=13, total=   0.0s
[CV] C=0.001, loss=squared_hinge, penalty=l2, random_state=13 ........


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  C=0.001, loss=squared_hinge, penalty=l2, random_state=13, total=   0.4s
[CV] C=0.001, loss=squared_hinge, penalty=l2, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l2, random_state=13, total=   0.5s
[CV] C=0.001, loss=squared_hinge, penalty=l2, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l2, random_state=13, total=   0.4s
[CV] C=0.001, loss=squared_hinge, penalty=l2, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l2, random_state=13, total=   0.4s
[CV] C=0.001, loss=squared_hinge, penalty=l2, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l2, random_state=13, total=   0.3s
[CV] C=0.001, loss=hinge, penalty=l1, random_state=13 ................
[CV] . C=0.001, loss=hinge, penalty=l1, random_state=13, total=   0.0s
[CV] C=0.001, loss=hinge, penalty=l1, random_state=13 ................
[CV] . C=0.001, loss=hinge, penalty=l1, random_state=13, total=   0.0s
[CV] C=0.001, loss=hinge, penalty=l1, rand

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   59.9s finished


GridSearchCV(estimator=LinearSVC(),
             param_grid={'C': [0.001, 0.01, 1],
                         'loss': ['squared_hinge', 'hinge'],
                         'penalty': ['l1', 'l2'], 'random_state': [13]},
             verbose=2)

In [57]:
print('LinearSVC Best parameters:', grid_cv.best_params_)
print('LinearSVC Best accruacy score:', grid_cv.best_score_)

LinearSVC Best parameters: {'C': 1, 'loss': 'hinge', 'penalty': 'l2', 'random_state': 13}
LinearSVC Best accruacy score: 0.6715714193437426


In [58]:
svc_svm_clf = LinearSVC(C=1, loss='hinge', penalty='l2', random_state=13)
svc_svm_clf.fit(X_train_tfidf_vect, y_train)

svc_train_pred = svc_svm_clf.predict(X_train_tfidf_vect)
svc_test_pred = svc_svm_clf.predict(X_test_tfidf_vect)

print('LinearSVC train accuracy score:', accuracy_score(y_train, svc_train_pred))
print('LinearSVC test accuracy score:', accuracy_score(y_test, svc_test_pred)) 

LinearSVC train accuracy score: 0.8779582260893333
LinearSVC test accuracy score: 0.6758381924198251


In [59]:
svc_svm_clf.decision_function(X_test_tfidf_vect) > 0

array([[ True, False, False, False, False],
       [False, False, False,  True, False],
       [False, False, False, False,  True],
       ...,
       [False, False, False, False, False],
       [False, False, False,  True, False],
       [False, False, False, False,  True]])

In [60]:
# 교차검증

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=13)

In [61]:
from sklearn.model_selection import cross_validate, cross_val_score

cross_validate(svc_svm_clf, X_train_tfidf_vect, y_train, scoring=None, cv=skf, return_train_score=True)

{'fit_time': array([6.16752315, 4.66517401, 4.4836328 , 4.46567297, 4.46346092,
        4.33517289, 4.37541413, 4.35532808, 4.36853671, 4.40511298]),
 'score_time': array([0.00145197, 0.00146794, 0.00146294, 0.00170588, 0.00146818,
        0.00142527, 0.00148392, 0.00146604, 0.00146031, 0.00145483]),
 'test_score': array([0.6745616 , 0.67205648, 0.68890913, 0.67061503, 0.68724374,
        0.67266515, 0.66970387, 0.67995444, 0.68633257, 0.676082  ]),
 'train_score': array([0.88284572, 0.88203584, 0.88064386, 0.88155797, 0.88122896,
        0.88100119, 0.88355731, 0.88049503, 0.88105181, 0.88360793])}

### SGDClassifier

In [62]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=13, loss='modified_huber')
sgd_clf.fit(X_train_tfidf_vect, y_train)

sgd_train_pred = sgd_clf.predict(X_train_tfidf_vect)
sgd_test_pred = sgd_clf.predict(X_test_tfidf_vect)

print('SGDClassifier train accuracy score:', accuracy_score(y_train, sgd_train_pred))
print('SGDClassifier test accuracy score:', accuracy_score(y_test, sgd_test_pred))

SGDClassifier train accuracy score: 0.8757715873630504
SGDClassifier test accuracy score: 0.6860422740524781


In [63]:
# 교차 검증

from sklearn.model_selection import cross_val_score

cross_val_score(sgd_clf, X_train_tfidf_vect, y_train, cv=5, scoring='accuracy')

array([0.69069582, 0.68409065, 0.67828266, 0.68462415, 0.68291572])

In [64]:
# 오차 행렬

from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train_tfidf_vect, y_train, cv=3)

In [65]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train, y_train_pred)

array([[6701,  624, 1182, 1525,  480],
       [ 588, 3643,  660,  751,  199],
       [ 819,  314, 6340, 1228,  521],
       [ 921,  398, 1193, 9187,  335],
       [ 741,  250,  936,  820, 3547]])

In [66]:
params = {
    'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3],
    'loss': ['log', 'modified_huber', 'hinge', 'squared_hinge', 'perceptron'],
    'penalty': ['l2', 'elasticnet'],
    'n_jobs': [-1],
    'random_state': [13],
}

grid_cv = GridSearchCV(sgd_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_cv.fit(X_train_tfidf_vect, y_train)

Fitting 3 folds for each of 80 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   44.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  3.6min finished


GridSearchCV(cv=3,
             estimator=SGDClassifier(loss='modified_huber', random_state=13),
             n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                                   1000.0],
                         'loss': ['log', 'modified_huber', 'hinge',
                                  'squared_hinge', 'perceptron'],
                         'n_jobs': [-1], 'penalty': ['l2', 'elasticnet'],
                         'random_state': [13]},
             scoring='accuracy', verbose=1)

In [67]:
print('SGDClassifier best parameters:', grid_cv.best_params_)
print('SGDClassifier best accuracy score:', grid_cv.best_score_)

SGDClassifier best parameters: {'alpha': 0.0001, 'loss': 'hinge', 'n_jobs': -1, 'penalty': 'l2', 'random_state': 13}
SGDClassifier best accuracy score: 0.6768785531684797


In [68]:
from sklearn.model_selection import cross_validate

cross_validate(sgd_clf, X_train_tfidf_vect, y_train, scoring=None, cv=skfold, return_train_score=True)

{'fit_time': array([0.52907896, 0.4407208 , 0.462749  , 0.46754003, 0.40408301]),
 'score_time': array([0.003371  , 0.00250316, 0.00320697, 0.00242591, 0.00259304]),
 'test_score': array([0.69069582, 0.68409065, 0.67828266, 0.68462415, 0.68291572]),
 'train_score': array([0.88918627, 0.88947099, 0.88930015, 0.88787974, 0.8856305 ])}

### RidgeClassifier

In [69]:
rd_clf = RidgeClassifier()
params = {
    'alpha': [0.01, 0.1, 0.5, 1.0],
    'normalize': [True, False],
    'max_iter': [100, 300],
    'random_state': [13]
}

grid_cv = GridSearchCV(rd_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_cv.fit(X_train_tfidf_vect, y_train)
print('RidgeClassifier best parameters:', grid_cv.best_params_)
print('RidgeClassifier best accuracy score:', grid_cv.best_score_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  1.2min finished


RidgeClassifier best parameters: {'alpha': 1.0, 'max_iter': 100, 'normalize': True, 'random_state': 13}
RidgeClassifier best accuracy score: 0.6520740179500645


In [70]:
rd_clf = RidgeClassifier(alpha=1.0, max_iter=100, normalize='False', random_state=13)
rd_clf.fit(X_train_tfidf_vect, y_train)

rd_train_pred = rd_clf.predict(X_train_tfidf_vect)
rd_test_pred = rd_clf.predict(X_test_tfidf_vect)
print('RidgeClassifier best train accuracy score:', accuracy_score(y_train, rd_train_pred))
print('RidgeClassifier best test accuracy score:', accuracy_score(y_test, rd_test_pred))

RidgeClassifier best train accuracy score: 0.8070519098922625
RidgeClassifier best test accuracy score: 0.6662718658892128


In [71]:
cross_validate(rd_clf, X_train_tfidf_vect, y_train, scoring=None, cv=skf, return_train_score=True)

{'fit_time': array([0.30153608, 0.30646992, 0.29524517, 0.29197907, 0.35633206,
        0.321141  , 0.31069326, 0.30892611, 0.30994701, 0.32381511]),
 'score_time': array([0.00180912, 0.002105  , 0.00191617, 0.00181198, 0.00201082,
        0.00224304, 0.00175691, 0.00173998, 0.00188708, 0.00187492]),
 'test_score': array([0.66545206, 0.65839217, 0.66977909, 0.65968109, 0.66993166,
        0.66355353, 0.65740319, 0.67312073, 0.66492027, 0.66218679]),
 'train_score': array([0.80990585, 0.81190524, 0.81066511, 0.81066991, 0.8103409 ,
        0.81153038, 0.81135323, 0.81051806, 0.81142915, 0.81218839])}

### LogisticRegression

In [72]:
clf = LogisticRegression()
params = {
    'C': [0.01, 0.1, 0.5, 1.0],
    'max_iter': [100, 200, 500],
}

grid_cv = GridSearchCV(clf, param_grid=params, scoring='accuracy', verbose=1, n_jobs=-1)
grid_cv.fit(X_train_tfidf_vect, y_train)
print('LogisticRegression best parameters:', grid_cv.best_params_)
print('LogisticRegression best accuracy score:', grid_cv.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  2.2min finished


LogisticRegression best parameters: {'C': 1.0, 'max_iter': 100}
LogisticRegression best accuracy score: 0.6820719149520125


In [73]:
lr_clf = LogisticRegression(C=1.0, max_iter=500)
lr_clf.fit(X_train_tfidf_vect, y_train)

lr_train_pred = lr_clf.predict(X_train_tfidf_vect)
lr_test_pred = lr_clf.predict(X_test_tfidf_vect)

print('LogisticRegression train accuracy score:', accuracy_score(y_train, lr_train_pred))
print('LogisticRegression test accuracy score:', accuracy_score(y_test, lr_test_pred))

LogisticRegression train accuracy score: 0.8671616973783113
LogisticRegression test accuracy score: 0.684402332361516


## 모델 결과

In [74]:
from sklearn.feature_extraction.text import CountVectorizer

abc = pd.merge(X_test, y_test, left_index = True, right_index=True)
abc['pred'] = sgd_test_pred

# 예측이 틀린 결과 -> 데이터 프레임 화
error_df = abc[abc['author'] != abc['pred']]

# CountVectorizer로 단어 갯수 카운트
cv = CountVectorizer(stop_words='english')   
cv_fit=cv.fit_transform(error_df['text'])    
word_list = cv.get_feature_names();    
count_list = cv_fit.toarray().sum(axis=0)

di = dict(zip(word_list,count_list))

# 예측이 틀린 결과 중 가장 많이 나온 단어 20개 정렬
miss = pd.DataFrame(list(di.items())).sort_values(by=1, ascending=False)
miss

,0,1
4642,odin,1790
4305,mind,61
4213,mean,56
2937,going,55
4401,morning,54
...,...,...
3388,hypothetical,1
3387,hypothesis,1
3386,hymn,1
3385,hydraulics,1


In [75]:
miss[miss[1] >= 50][0].tolist()

['odin',
 'mind',
 'mean',
 'going',
 'morning',
 'mother',
 'sat',
 'ill',
 'leave',
 'home',
 'poor']

In [76]:
# from catboost import CatBoostClassifier

# cb = CatBoostClassifier(silent=True, random_state=13, n_estimators=300).fit(X_train_tfidf_vect, y_train)
# accuracy_score(y_train, cb.predict(X_train_tfidf_vect))
# accuracy_score(y_test, cb.predict(X_test_tfidf_vect))

In [77]:
# accuracy_score(y_train, cb.predict(X_train_tfidf_vect))

## 모델 검증

In [78]:
t_models = []

t_models.append(('MultinomialNB', MultinomialNB(alpha=0.5, fit_prior='True')))
t_models.append(('LinearSVC', LinearSVC(C=1, loss='hinge', penalty='l2', random_state=13)))
t_models.append(('SGDClassifier', SGDClassifier(alpha=0.0001, loss='modified_huber', n_jobs=-1, penalty='l2', random_state=13)))
t_models.append(('RidgeClassifier', RidgeClassifier(alpha=1.0, max_iter=100, normalize='False', random_state=13)))
t_models.append(('LogisticRegression', LogisticRegression(C=1.0, max_iter=500)))

In [79]:
from sklearn.metrics import accuracy_score

train_score = []
test_score = []
names = []

for name, model in t_models:
    clf = model
    clf.fit(X_train_tfidf_vect, y_train)
    
    train_pred = clf.predict(X_train_tfidf_vect)
    test_pred = clf.predict(X_test_tfidf_vect)
    
    names.append(name)
    train_score.append(accuracy_score(y_train, train_pred))
    test_score.append(accuracy_score(y_test, test_pred))

In [80]:
result = pd.DataFrame({'model name': names,
                       'train score': train_score,
                       'test score': test_score})
result['diff'] = result['train score'] - result['test score']
result.round(2).sort_values(by='train score', ascending=False).reset_index(drop=True)

,model name,train score,test score,diff
0,LinearSVC,0.88,0.68,0.20
1,SGDClassifier,0.88,0.69,0.19
2,LogisticRegression,0.87,0.68,0.18
3,RidgeClassifier,0.81,0.67,0.14
4,MultinomialNB,0.78,0.71,0.07
